

1. Model name: 
2. Domain: Multi-Criteria Recommender System (MCRS)
2. Code writer: Jin-Duk Park (github: https://github.com/jindeok, homepage: https://jindeok.github.io/jdpark/)

In [1]:
import random
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from scipy.sparse import csr_matrix, save_npz, load_npz

random.seed(2022)
np.random.seed(2022)

c:\Users\CSE\anaconda3\envs\GL\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [160]:
import os

current_directory = os.getcwd()
print("Current Directory:", current_directory)
# Configuration
dataset = "BA" # BA, YM, TA
path = f"{current_directory}/dataset/{dataset}"
cri_dict =  {"TA":8, "YM":5, "BA":5}

R_tr = []
R_ts = []

# Read csr matrix
for idx in range(cri_dict[dataset]):
    R_tr.append(load_npz(path + f'/{dataset}_tr_{idx}.npz'))
    R_ts.append(load_npz(path + f'/{dataset}_ts_{idx}.npz'))


n_users = R_tr[0].shape[0]
n_items = R_tr[0].shape[1]


Current Directory: i:\다른 컴퓨터\집컴퓨터\DataScience\MC_GF\Code


In [161]:
import numpy as np
from scipy.sparse import csr_matrix,vstack

# Example csr_matrices: r1, r2, ..., rc
# Each matrix should be a csr_matrix

# Calculate the correlation between non-zero entries
def calculate_row_correlations(matrix1, matrix2):
    num_rows = matrix1.shape[0]
    correlations = np.zeros(num_rows)
    
    for row in range(num_rows):
        nz_indices1 = matrix1.indices[matrix1.indptr[row]:matrix1.indptr[row+1]]
        nz_indices2 = matrix2.indices[matrix2.indptr[row]:matrix2.indptr[row+1]]
        
        common_indices = np.intersect1d(nz_indices1, nz_indices2)
        
        nz_values1 = matrix1.data[matrix1.indptr[row]:matrix1.indptr[row+1]][np.searchsorted(nz_indices1, common_indices)]
        nz_values2 = matrix2.data[matrix2.indptr[row]:matrix2.indptr[row+1]][np.searchsorted(nz_indices2, common_indices)]
        
        if len(common_indices) > 0:
            correlation = np.corrcoef(nz_values1, nz_values2)[0, 1]
            correlations[row] = correlation
    
    return correlations

# Calculate correlations between each row of r1 and each row of r2, r3, ..., rc
row_correlations = []
for matrix in R_tr[1:]:
    correlations = calculate_row_correlations(R_tr[0], matrix)
    row_correlations.append(correlations)

# Print the calculated row correlations
for idx, correlations in enumerate(row_correlations, start=2):
    print(f"Row correlations between r1 and r{idx}:", correlations)

corr_arr = np.array(row_correlations)
corr_arr = corr_arr.T


c:\Users\CSE\anaconda3\envs\GL\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\Users\CSE\anaconda3\envs\GL\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
c:\Users\CSE\anaconda3\envs\GL\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\CSE\anaconda3\envs\GL\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\CSE\anaconda3\envs\GL\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


Row correlations between r1 and r2: [0.39239648 0.6123417  0.2092888  ... 0.88345221 0.88388348 0.56685302]
Row correlations between r1 and r3: [0.63430998 0.75011656 0.36566308 ... 0.76827333 0.62749502 0.67687019]
Row correlations between r1 and r4: [0.52054356 0.79724786 0.46895489 ...        nan 0.80211118 0.53787716]
Row correlations between r1 and r5: [0.92935649 0.79895209 0.60626965 ... 0.81150267 0.57282196 0.63789881]


In [162]:
# 모든 값이 NaN인 행 개수 세기
all_nan_rows = np.all(np.isnan(corr_arr), axis=1)
all_nan_count = np.count_nonzero(all_nan_rows)

print("모든 값이 NaN인 행 개수:", all_nan_count)

모든 값이 NaN인 행 개수: 137


In [163]:
#Calculate Correlations (user-specific preference of criterion)

# 딕셔너리 초기화
pref_map = {}

# 각 행에 대해서 가장 큰 값의 열 인덱스를 찾아서 딕셔너리에 추가
for row_index, row in enumerate(corr_arr):
    if np.any(~np.isnan(row)):
        # NaN이 아닌 값이 있는 경우에만 처리
        max_col_index = np.nanargmax(row)  # nan을 무시하고 가장 큰 값의 열 인덱스 찾기
    else:
        max_col_index = 0
    pref_map[row_index] = max_col_index

print(pref_map)

{0: 3, 1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 6: 3, 7: 3, 8: 3, 9: 3, 10: 3, 11: 0, 12: 3, 13: 3, 14: 3, 15: 3, 16: 3, 17: 3, 18: 3, 19: 2, 20: 3, 21: 3, 22: 3, 23: 3, 24: 3, 25: 2, 26: 3, 27: 3, 28: 3, 29: 3, 30: 3, 31: 3, 32: 3, 33: 3, 34: 3, 35: 3, 36: 3, 37: 3, 38: 3, 39: 3, 40: 3, 41: 3, 42: 3, 43: 3, 44: 3, 45: 3, 46: 3, 47: 3, 48: 2, 49: 3, 50: 2, 51: 3, 52: 3, 53: 3, 54: 3, 55: 3, 56: 3, 57: 3, 58: 3, 59: 3, 60: 3, 61: 3, 62: 3, 63: 3, 64: 3, 65: 3, 66: 3, 67: 3, 68: 3, 69: 3, 70: 3, 71: 3, 72: 2, 73: 3, 74: 3, 75: 3, 76: 3, 77: 3, 78: 3, 79: 3, 80: 3, 81: 3, 82: 3, 83: 2, 84: 2, 85: 3, 86: 3, 87: 2, 88: 3, 89: 3, 90: 3, 91: 2, 92: 3, 93: 3, 94: 3, 95: 3, 96: 2, 97: 3, 98: 3, 99: 2, 100: 3, 101: 3, 102: 3, 103: 3, 104: 3, 105: 3, 106: 3, 107: 3, 108: 3, 109: 3, 110: 2, 111: 3, 112: 3, 113: 3, 114: 3, 115: 3, 116: 3, 117: 2, 118: 3, 119: 2, 120: 3, 121: 3, 122: 2, 123: 2, 124: 3, 125: 3, 126: 3, 127: 3, 128: 3, 129: 3, 130: 3, 131: 3, 132: 3, 133: 3, 134: 3, 135: 3, 136: 2, 137: 2, 138: 

In [164]:
def get_norm_adj(alpha, adj_mat):
    rowsum = np.array(adj_mat.sum(axis=1))
    d_inv = np.power(rowsum, -alpha).flatten()
    d_inv[np.isinf(d_inv)] = 0.
    d_mat = sp.diags(d_inv)
    norm_adj = d_mat.dot(adj_mat)
    colsum = np.array(adj_mat.sum(axis=0))
    d_inv = np.power(colsum, alpha-1).flatten()
    d_inv[np.isinf(d_inv)] = 0.
    d_mat = sp.diags(d_inv)
    d_mat_i = d_mat
    d_mat_i_inv = sp.diags(1/d_inv)
    d_mat_i_inv.data = np.nan_to_num(d_mat_i_inv.data)
    norm_adj = norm_adj.dot(d_mat)
    return norm_adj.tocsc()

In [165]:
# Cri 별 P행렬들 미리 계산해놓기
# Normalization 계산
import scipy.sparse as sp

Ps = []

for R in R_tr:
    R = get_norm_adj(0.5, R)
    Ps.append(R.T @ R)

C:\Users\CSE\AppData\Local\Temp\ipykernel_15932\2580273546.py:8: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(colsum, alpha-1).flatten()
C:\Users\CSE\AppData\Local\Temp\ipykernel_15932\2580273546.py:12: RuntimeWarning: divide by zero encountered in true_divide
  d_mat_i_inv = sp.diags(1/d_inv)


In [166]:
# User-wise Prediction, w/ preference mapping

S = []

for u in range(n_users):
    S.append(R_tr[pref_map[u]][u] @ Ps[pref_map[u]])
S_sparse = vstack(S)
prediction = S_sparse.todense()

In [169]:
# User-wise Prediction, w/o preference mapping

S = []

for u in range(n_users):
    S.append(R_tr[0][u] @ Ps[0])
S_sparse = vstack(S)
prediction = S_sparse.todense()

In [170]:
# Evaluation functions

def top_k(S, k=1):
    """
    S: scores, numpy array of shape (M, N) where M is the number of source nodes,
        N is the number of target nodes
    k: number of predicted elements to return
    """
    top = np.argsort(-S)[:, :k]
    result = np.zeros(S.shape)
    for idx, target_elms in enumerate(top):
        for elm in target_elms:
            result[idx, elm] = 1

    return result, top
    
def precision_k(topk, gt, k):
    '''
    topk, gt: (UXI) array
    k: @k measurement
    '''
    return np.multiply(topk, gt).sum() / (k * len(gt))


def recall_k(topk, gt, k):
    '''
    topk, gt: (UXI) array
    k: @k measurement
    '''
    return np.multiply(topk, gt).sum() / gt.sum()

def ndcg_k_(rels, rels_ideal, gt):
    '''
    rels: sorted top-k arr
    rels_ideal: sorted top-k ideal arr
    '''
    dcg =0; idcg = 0
    for row, (row_rels, row_rels_i) in enumerate(zip(rels, rels_ideal)):
        for i, (rel, rel_i) in enumerate(zip(row_rels, row_rels_i)):
            if gt[row,rel[0,i]] == 1:
                if i==0:
                    dcg += 1
                else:
                    dcg += 1/np.log2(i+1)
            if gt[row,rel_i[0,i]] == 1:
                if i==0:
                    idcg += 1
                else:
                    idcg += 1/np.log2(i+1)
    return dcg/idcg



In [171]:
# Start evaluation procedure for multi-criteria recommender system.
import copy

# Prediction, Put every test set at once ( * if memory issue exists, slice it with batch instances * )
results = prediction
# seen items -> -inf (test should be done among the unseen pool)
results = results + (-99999)*R_tr[0]
# (UXI) topk_pr_mat, gt_mat
top_10_mat, rel10 = top_k(results, k = 10)
top_5_mat, rel5 = top_k(results, k = 5)

# Processing gt
gt_mat = R_ts[0].todense()
gt_mat[gt_mat<3] = 0
gt_mat[gt_mat>=3] = 1
_, i_rel10 =top_k(gt_mat, k = 10)
_, i_rel5 =top_k(gt_mat, k = 5)

# measure ranking-based evaluations
print(f"Precision@5: {precision_k(top_5_mat, gt_mat, 5):.4f}, Precision@10: {precision_k(top_10_mat, gt_mat, 10):.4f} \n")
print(f"Recall@5: {recall_k(top_5_mat, gt_mat, 5):.4f}, Recall@10: {recall_k(top_10_mat, gt_mat, 10):.4f} \n")
print(f"NDCG@5: {ndcg_k(rel5, i_rel5, gt_mat):.4f} NDCG@10: {ndcg_k(rel10, i_rel10, gt_mat):.4f} \n")

Precision@5: 0.1031, Precision@10: 0.0870 

Recall@5: 0.0426, Recall@10: 0.0718 

NDCG@5: 0.1492 NDCG@10: 0.1492 



In [ ]:
## Visualization codes


import matplotlib.pyplot as plt
TA_cri = ["value",	"service",	"rooms",	"location",	"cleanliness",	"checkin",	"business"]
BA_cri = ['Aroma', 'Appearance', 'Palate', 'Taste']
YM_cri = ['Visuals',	'Direction',	'Story',	'Acting']
# Example data: replace with your own data
selected_row_index = 
selected_row = corr_arr[selected_row_index]

# Column labels for X axis
column_labels = YM_cri

# Create a bar chart
bar_colors = ['blue', 'green', 'orange', 'red']
plt.figure(figsize=(5 ,6))
bars = plt.bar(column_labels, selected_row, color=bar_colors)
plt.xlabel('Criterion')
plt.ylabel('Corr.')
plt.title(f'Corr. with overall, user {selected_row_index}')
#plt.ylim(-1, 1)  # Set the Y axis range to 0-1
plt.grid(True)

# Add text labels above each bar
for bar in bars:
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.01,
             f'{bar.get_height():.2f}', ha='center', va='bottom')

plt.show()
